In [1]:
import os
import sys
import pandas as pd
sys.path.append('/path/to/repo/performance_evaluation/development_code/')
import modeval

This notebook assumes a sinple 5CV grid search was performed as (e.g. such as specified in https://app.box.com/file/565474267762) <br>

In [ ]:
# pandas v0.25 (lower than 1 is enough)

In [2]:
pd.__version__

'0.25.3'

In [ ]:
# specify sparsechem's folder containing HP grid search results
result_dir = 'results/'
model_dir = 'models/'

### SELECT BEST FROM JSON FILES

In [ ]:
# use the following if performance reports in *.json files 
# collect performance metrics from conf file (aggregate form) 
conf_df = modeval.perf_from_json(result_dir, aggregate=True)

In [ ]:
# melt: in conf files, there is only auc_pr and auc_roc
conf_dfm = modeval.melt_perf(conf_df, perf_metrics=['roc_auc_score_agg', 'auc_pr_agg', 'avg_prec_score_agg', 'max_f1_score_agg', 'kappa_agg', 'logloss_agg']) 

# Find out the best hyperparameters
best_hp = modeval.best_hyperparam(conf_dfm)

In [ ]:
# show the hyperparameters 
wanted_score='auc_pr_agg'
best_hp.loc[best_hp['score_type']==wanted_score]

### SELECT BEST FROM CONF.NPY FILES (OLD SPARSECHEM VERSION)

In [3]:
# use the following if performance reports in *.conf.npy files 
# collect performance metrics from conf file (aggregate form) 
conf_df = modeval.perf_from_conf(model_dir, aggregate=True)

In [3]:
# melt: in conf files, there is only auc_pr and auc_roc (no kappa, f1, ...) 
conf_dfm = modeval.melt_perf(conf_df, perf_metrics=['auc_pr_va_mean', 'auc_va_mean']) 

# Find out the best hyperparameters
best_hp = modeval.best_hyperparam(conf_dfm)

In [ ]:
# show the hyperparameters 
wanted_score='auc_pr_va_mean'
best_hp.loc[best_hp['score_type']==wanted_score]